Using geocoders in Python
=============

A lot of the geographic functionality that we've seen in Google Maps, or OpenStreetMap, can be got at programmatically! This can be really useful for getting information about a place, even when you don't want to show it on a map.

Today we will look at the GeoPy library, which gives us a consistent interface to a bunch of services including Google maps, GeoNames, Bing, Baidu, OpenStreetMap / Nominatim, and others.

Go to a command / terminal window and type

    pip install geopy
    
or

    pip3 install geopy
    
if you are on Linux.

In [ ]:
import geopy

Place text search
------------

You have some place name, and you want to find out where it is. We can do this using a variety of services, and [Geopy's documentation](http://geopy.readthedocs.org/en/1.10.0/) tells us how. First we will use Nominatim, which is the database behind OpenStreetMap.

In [ ]:
from geopy.geocoders import Nominatim
geocoder = Nominatim()
location = geocoder.geocode("Constantinople")
print(location.raw)
print(location.latitude, location.longitude)
print("Address is %s" % location.address)

If we also pass the parameter `exactly_one` and set it to `False`, then we will get every place that matches the search term, instead of just the most likely place.

In [ ]:
locationlist = geocoder.geocode("Zurich", exactly_one=False)
print([x.address for x in locationlist])

That funny-looking thing in the print statement above is a *list comprehension*. It is a short and sweet way of saying this:

In [ ]:
addresses = []
for x in locationlist:
    addresses.append(x.address)
print(addresses)

Another geocoding service that is often useful is called GeoNames. It is a database that collects the names, in different languages and different times, of various places. So if you know what a place was called in Hungarian in the seventeenth century but Google Maps has never heard of it, you can try Geonames, which might tell you its current Romanian name!

In [ ]:
from geopy.geocoders import GeoNames
geocoder = GeoNames(username="aurum")
location = geocoder.geocode("Edessa", exactly_one=False)
for x in location:
    print("Found a location with data: %s" % x.raw)


Of course Google itself is also an option. Google geolocation requires an API key, which is essentially a special password that is associated with your user account, to use its web API. This is because Google sets limits to how much you can use it for free. The limits should plenty for a normal person's use, though.

For the time being, you can use the API key that I have put in ILIAS as long as you are on the network of the Uni Bern. If you are going to do your own work with Maps, though, then you should go to http://developers.google.com/ and sign up as a developer. You'll then need to create a project, and in that project go to 'Google APIs' -> "Google Maps Geocoding API" and enable it. You'll then need to go to 'Credentials' to make your API key. If you need further help with the options, then talk to me!

In [ ]:
from geopy.geocoders import GoogleV3
geocoder = GoogleV3()
location = geocoder.geocode("Zürich")
print(location.raw)

Using an API directly
------------

The Geopy library is very useful in that it lets you do the same job using a variety of services. However, it doesn't handle everything that a particular service might offer. Google has something else, called the "Places API", that gives information not just about typical geographic locations but also about businesses, monuments, and so on. Here is an example of how this is used - if you want to do this with your own API key, you will need to enable the 'Google Places API Web Service' as well, in the online developer console!



In [ ]:
import requests

google_api_key = 'KEY GOES HERE'

places_search_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
search_params = {
    'query': 'Länggass Stübli',
    'key': google_api_key,
    'language': 'en'
    }

r = requests.get( places_search_url, params=search_params)
search_result = r.json()   # See what we got
print(search_result)

Once we have successfully looked up a place on Google, we will have an ID for it. This is Google's way of distinguishing between places of the same name, so that we know we have the right one. We can use that ID to get information about a place we have already looked up, but for this we have to leave Geopy behind and use the API directly!

In [ ]:
places_details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
detail_params = {
    'key': google_api_key,
    'placeid': search_result['results'][0]['place_id'],
    'language': 'en'
}

r = requests.get( places_details_url, params=detail_params)
r.json()  # See what we got

Now let's look up a series of places! We'll store our results in places_found, for each place that we find.

Exporting our data to CSV
--------

One thing we can do with the Places API is to look up a bunch of places, get their latitude and longitude or their canonical names, and put those into a big spreadsheet for use elsewhere (or even for importing into Google Maps to make a map!)

Let's look up a bunch of place names so that we can put their information into a CSV file that we will make. We have four search terms, and for each one we'll see if we get a result; if we do, it will go into the `places_found` dictionary that we will use below.

In [ ]:
places_to_lookup = ['Moskva', 'Venice', 'Rosslyn Chapel', 'Cantabrigia']
places_found = {}

geocoder = GoogleV3()
for p in places_to_lookup:
    myresult = geocoder.geocode(p)
    if p is not None:
        print("Found information for %s" % p)
        places_found[p] = myresult

The easiest way to make something like a spreadsheet in a computer program is to use CSV, which stands for *comma separated values*. That is what we used earlier to get our UK fat supply data into our map. Python has a built-in module for this, and we use it like this to make a CSV file.

In [ ]:
import csv

f = open('../lessondata/myplaces.csv', 'w', newline='', encoding='utf-8')
writer = csv.writer(f)
# First, write our column headers!
writer.writerow(['Place name', 'Address', 'Latitude', 'Longitude'])


Now we have an open file in our data directory called 'myplaces.csv', and we have written one row to it. If you were to close the filehandle now and look at the file, you would see that it looks like this:

    Place name,Address,ID,Latitude,Longitude
    
But we won't close the file yet, because we want to write each of our places into its row.

In [ ]:
for p in places_found.keys():
    location = places_found[p]
    writer.writerow([p, location.address, location.latitude, location.longitude])

f.close()  # Always close what you open, if you didn't use 'with'!
    

Now we can make sure the file is there and has what we expect!

In [ ]:
with open('../lessondata/myplaces.csv', encoding='utf-8') as f:
    data = f.read()
    
print(data)

And now you can find that file on your computer (it should be in the same folder as this notebook) and use it for processing anywhere you like, including putting back into Google Maps if you so choose.